In [ ]:
!pip install einops
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux20

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from einops import rearrange
from math import sqrt

### DConv


In [ ]:
class DConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, pad=0):
        super(DConvBlock, self).__init__()

        self.conv2d_30 = nn.Conv2d(in_channels, out_channels, kernel_size=(30, 30), stride=1, padding=pad)
        self.conv2d_60 = nn.Conv2d(out_channels, out_channels, kernel_size=(60, 60), stride=1, padding=pad)
        self.norm = nn.BatchNorm2d(out_channels)
        self.conv1x1_1 = nn.Conv2d(out_channels, out_channels, kernel_size=1)
        self.gelu = nn.GELU()
        self.conv1x1_2 = nn.Conv2d(out_channels, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.conv2d_30(x)
        x = self.conv2d_60(x)
        x = self.norm(x)
        x = self.conv1x1_1(x)
        x = self.gelu(x)
        x = self.conv1x1_2(x)

        return x

### SWIN from https://github.com/HuCaoFighting/Swin-Unet/blob/main/networks/swin_transformer_unet_skip_expand_decoder_sys.py#L282

In [ ]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size

    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows


def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image

    Returns:
        x: (B, H, W, C)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x


class WindowAttention(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.

    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):

        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 2*Wh-1 * 2*Ww-1, nH

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

    def extra_repr(self) -> str:
        return f'dim={self.dim}, window_size={self.window_size}, num_heads={self.num_heads}'

    def flops(self, N):
        # calculate flops for 1 window with token length of N
        flops = 0
        # qkv = self.qkv(x)
        flops += N * self.dim * 3 * self.dim
        # attn = (q @ k.transpose(-2, -1))
        flops += self.num_heads * N * (self.dim // self.num_heads) * N
        #  x = (attn @ v)
        flops += self.num_heads * N * N * (self.dim // self.num_heads)
        # x = self.proj(x)
        flops += N * self.dim * self.dim
        return flops


class SwinTransformerBlock(nn.Module):
    r""" Swin Transformer Block.

    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resulotion.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Module, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, dim, input_resolution, num_heads, window_size=8, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        if self.shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x

        # partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # nW*B, window_size*window_size, C

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C

        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x = x.view(B, H * W, C)

        # FFN
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x

    def extra_repr(self) -> str:
        return f"dim={self.dim}, input_resolution={self.input_resolution}, num_heads={self.num_heads}, " \
               f"window_size={self.window_size}, shift_size={self.shift_size}, mlp_ratio={self.mlp_ratio}"

    def flops(self):
        flops = 0
        H, W = self.input_resolution
        # norm1
        flops += self.dim * H * W
        # W-MSA/SW-MSA
        nW = H * W / self.window_size / self.window_size
        flops += nW * self.attn.flops(self.window_size * self.window_size)
        # mlp
        flops += 2 * H * W * self.dim * self.dim * self.mlp_ratio
        # norm2
        flops += self.dim * H * W
        return flops


### Patch Merging from https://github.com/HuCaoFighting/Swin-Unet/blob/main/networks/swin_transformer_unet_skip_expand_decoder_sys.py#L282

In [ ]:
class PatchMerging(nn.Module):
    r""" Patch Merging Layer.

    Args:
        input_resolution (tuple[int]): Resolution of input feature.
        dim (int): Number of input channels.
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, input_resolution, dim, norm_layer=nn.LayerNorm):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, dim, bias=False)
        self.norm = norm_layer(4 * dim)

    def forward(self, x):
        """
        x: B, H*W, C
        """
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."

        x = x.view(B, H, W, C)

        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
        x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
        x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
        x = torch.cat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        x = x.view(B, -1, 4 * C)  # B H/2*W/2 4*C

        x = self.norm(x)
        x = self.reduction(x)

        return x

    def extra_repr(self) -> str:
        return f"input_resolution={self.input_resolution}, dim={self.dim}"

    def flops(self):
        H, W = self.input_resolution
        flops = H * W * self.dim
        flops += (H // 2) * (W // 2) * 4 * self.dim * self.dim
        return flops

### Patch Expansion from https://github.com/HuCaoFighting/Swin-Unet/blob/main/networks/swin_transformer_unet_skip_expand_decoder_sys.py#L434

In [ ]:
class PatchExpand(nn.Module):
    def __init__(self, input_resolution, dim, dim_scale=2, norm_layer=nn.LayerNorm):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.expand = nn.Linear(dim, dim * (dim_scale ** 2), bias=False) if dim_scale==2 else nn.Identity()
        self.norm = norm_layer(dim)

    def forward(self, x):
        """
        x: B, H*W, C
        """
        H, W = self.input_resolution
        x = self.expand(x)

        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        x = x.view(B, H, W, C)
        x = rearrange(x, 'b h w (p1 p2 c)-> b (h p1) (w p2) c', p1=2, p2=2, c=C//4)

        x = x.view(B,-1,C//4)

        x= self.norm(x)

        return x

### CSU-NET Architecture



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CSUNet(nn.Module):
    def __init__(self):
        super(CSUNet, self).__init__()
        self.DConv1 = DConvBlock(in_channels=3, out_channels=64)
        self.DConv2 = DConvBlock(in_channels=3, out_channels=64)
        self.DConv3 = DConvBlock(in_channels=64, out_channels=64)

        self.encoder1 = SwinTransformerBlock(dim=64, input_resolution=(392, 392), num_heads=4)
        self.encoder2 = SwinTransformerBlock(dim=64, input_resolution=(392, 392), num_heads=4)
        self.encoder3 = SwinTransformerBlock(dim=64, input_resolution=(304, 304), num_heads=4)

        self.patch_merging = PatchMerging(input_resolution=(392, 392), dim=64)
        self.patch_merging2 = PatchMerging(input_resolution=(304, 304), dim=64)

        self.decoder_swin_block1 = SwinTransformerBlock(dim=64, input_resolution=(340, 340), num_heads=4, window_size=5)
        self.decoder_swin_block2 = SwinTransformerBlock(dim=64, input_resolution=(380, 380), num_heads=4, window_size=5)
        self.decoder_swin_block3 = SwinTransformerBlock(dim=64, input_resolution=(425, 425), num_heads=4, window_size=5)

        self.upsampler = nn.Upsample(scale_factor=1.25, mode='linear', align_corners=False)

        self.reduce_conv1 = nn.Conv2d(64, 64, kernel_size=1)
        self.reduce_conv2 = nn.Conv2d(64, 64, kernel_size=1)
        self.reduce_conv3 = nn.Conv2d(64, 64, kernel_size=1)

        self.final_conv = nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1)

    def forward(self, x):
        print(f'Input: {x.shape}')

        cnn_features1 = self.DConv1(x)

        cnn_features1 = cnn_features1.permute(0, 2, 3, 1)
        B, H, W, C = cnn_features1.shape
        cnn_features1 = cnn_features1.view(B, H * W, C)

        cnn_features2 = self.DConv2(x)

        cnn_features2 = cnn_features2.permute(0, 2, 3, 1)
        B, H, W, C = cnn_features2.shape
        cnn_features2 = cnn_features2.view(B, H * W, C)

        encoder1_output = self.encoder1(cnn_features1)

        patch_merging_output = self.patch_merging(encoder1_output)

        padding_needed = cnn_features2.shape[1] - patch_merging_output.shape[1]
        patch_merging_output = F.pad(patch_merging_output, (0, 0, 0, padding_needed))

        combined_features2 = patch_merging_output + cnn_features2

        encoder2_output = self.encoder2(combined_features2)

        patch_merging_output2 = self.patch_merging(encoder2_output)

        B, L, C = cnn_features1.shape
        H = W = int(L ** 0.5)
        cnn_features1 = cnn_features1.view(B, H, W, C)
        cnn_features1 = cnn_features1.permute(0, 3, 1, 2)

        dconv3_output = self.DConv3(cnn_features1)

        dconv3_output = dconv3_output.permute(0, 2, 3, 1)
        B, H, W, C = dconv3_output.shape
        dconv3_output = dconv3_output.view(B, H * W, C)

        padding_needed = dconv3_output.shape[1] - patch_merging_output2.shape[1]
        patch_merging_output2 = F.pad(patch_merging_output2, (0, 0, 0, padding_needed))

        combined_features3 = patch_merging_output2 + dconv3_output

        encoder3_output = self.encoder3(combined_features3)

        patch_merging_output3 = self.patch_merging2(encoder3_output)

        decoder_output1 = torch.cat([patch_merging_output3, encoder3_output], dim=1)
        decoder_output1 = decoder_output1.transpose(1, 2)
        decoder_output1 = F.interpolate(decoder_output1, 115600, mode='linear', align_corners=False)
        decoder_output1 = decoder_output1.transpose(1, 2)

        B, L, C = decoder_output1.shape
        H = W = int(L ** 0.5)
        decoder_output1 = decoder_output1.transpose(1, 2).view(B, 64, H, W)

        decoder_output1 = self.reduce_conv1(decoder_output1)
        decoder_output1 = decoder_output1.permute(0, 2, 3, 1)
        B, W, H, C = decoder_output1.shape
        decoder_output1 = decoder_output1.view(B, W*H, C)
        decoder_output1 = self.decoder_swin_block1(decoder_output1)
        decoder_output1 = decoder_output1.transpose(1, 2)

        upsampled_output1 = self.upsampler(decoder_output1)
        upsampled_output1 = F.interpolate(upsampled_output1, 144400, mode='linear', align_corners=False)
        upsampled_output1 = upsampled_output1.transpose(1, 2)
        B, L, C = upsampled_output1.shape
        H = W = int(L ** 0.5)
        upsampled_output1 = upsampled_output1.transpose(1, 2).view(B, 64, H, W)

        decoder_output2 = self.reduce_conv2(upsampled_output1)
        decoder_output2 = decoder_output2.permute(0, 2, 3, 1)
        B, W, H, C = decoder_output2.shape
        decoder_output2 = decoder_output2.view(B, W*H, C)
        decoder_output2 = self.decoder_swin_block2(decoder_output2)
        decoder_output2 = decoder_output2.transpose(1, 2)

        upsampled_output2 = self.upsampler(decoder_output2)
        upsampled_output2 = F.interpolate(upsampled_output2, 180625, mode='linear', align_corners=False)
        upsampled_output2 = upsampled_output2.transpose(1, 2)
        B, L, C = upsampled_output2.shape
        H = W = int(L ** 0.5)
        upsampled_output2 = upsampled_output2.transpose(1, 2).view(B, 64, H, W)

        decoder_output3 = self.reduce_conv3(upsampled_output2)
        decoder_output3 = decoder_output3.permute(0, 2, 3, 1)
        B, W, H, C = decoder_output3.shape
        decoder_output3 = decoder_output3.view(B, W*H, C)
        decoder_output3 = self.decoder_swin_block3(decoder_output3)
        decoder_output3 = decoder_output3.transpose(1, 2)

        upsampled_output3 = self.upsampler(decoder_output3)
        upsampled_output3 = F.interpolate(upsampled_output3, 230400, mode='linear', align_corners=False)
        upsampled_output3 = upsampled_output3.transpose(1, 2)
        B, L, C = upsampled_output3.shape
        H = W = int(L ** 0.5)
        upsampled_output3 = upsampled_output3.transpose(1, 2).view(B, 64, H, W)

        out = self.final_conv(upsampled_output3)

        return out


### Data Loader

In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        image = image.resize((480, 480), Image.BILINEAR)
        mask = mask.resize((480, 480), Image.NEAREST)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

### Loss Function

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, inputs, targets):
        # Apply softmax to get probabilities
        inputs = torch.sigmoid(inputs)

        inputs = inputs.view(-1)
        targets = targets.view(-1)

        # Calculate intersection and union
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()

        # Calculate Dice coefficient
        dice = (2.0 * intersection + self.smooth) / (total + self.smooth)

        # Return Dice loss
        return 1.0 - dice

In [ ]:
class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super(CombinedLoss, self).__init__()
        self.dice_loss = DiceLoss()
        self.alpha = alpha

    def forward(self, pred, target):
        dice = self.dice_loss(pred, target)
        ce = F.cross_entropy(pred, target)

        loss = (self.alpha * dice) + ((1 - self.alpha) * ce)
        return loss

### Train

In [ ]:
 transform = transforms.ToTensor()
image_dir = "/content/drive/MyDrive/Colab Notebooks/Colon_Cancer/Kvasir-SEG/images"
mask_dir = "/content/drive/MyDrive/Colab Notebooks/Colon_Cancer/Kvasir-SEG/masks"
dataset = SegmentationDataset(image_dir, mask_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Move model to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CSUNet().to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CombinedLoss(alpha=0.7)

# Training loop
num_epochs = 10  # Adjust as needed
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    iter = 0

    for images, masks in dataloader:
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, masks)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        iter+=1
        print(f"Iteration {iter} of epoch {epoch+1}\nLoss: {loss.item()}")

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader)}")
    print("*"*20)


/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Input: torch.Size([1, 3, 480, 480])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Iteration 1 of epoch 1
Loss: 0.382997065782547
Input: torch.Size([1, 3, 480, 480])
Iteration 2 of epoch 1
Loss: 0.6339851021766663
Input: torch.Size([1, 3, 480, 480])
Iteration 3 of epoch 1
Loss: 0.608011782169342
Input: torch.Size([1, 3, 480, 480])
Iteration 4 of epoch 1
Loss: 0.29715999960899353
Input: torch.Size([1, 3, 480, 480])
Iteration 5 of epoch 1
Loss: 0.46872568130493164
Input: torch.Size([1, 3, 480, 480])
Iteration 6 of epoch 1
Loss: 0.36791104078292847
Input: torch.Size([1, 3, 480, 480])
Iteration 7 of epoch 1
Loss: 0.604047417640686
Input: torch.Size([1, 3, 480, 480])
Iteration 8 of epoch 1
Loss: 0.5299990177154541
Input: torch.Size([1, 3, 480, 480])
Iteration 9 of epoch 1
Loss: 0.651130199432373
Input: torch.Size([1, 3, 480, 480])
Iteration 10 of epoch 1
Loss: 0.3181731700897217
Input: torch.Size([1, 3, 480, 480])
Iteration 11 of epoch 1
Loss: 0.25502485036849976
Input: torch.Size([1, 3, 480, 480])
Iteration 12 of epoch 1
Loss: 0.45409345626831055
Input: torch.Size([1, 3,